In [3]:
## Callin Switzer
## Capture Images from pt grey camera

In [4]:
# setup arduino
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

import datetime

import PyCapture2 as fc2
import sys

import time

import os
import peakutils 
import msvcrt
import winsound
import shutil
import pandas as pd
import re

import skimage.io as io

In [5]:
print("last update: "  +  str(datetime.datetime.now()))

last update: 2018-01-24 11:27:26.849979


In [6]:
# capture image with point grey camera
bus = fc2.BusManager()
numCams = bus.getNumOfCameras()
print("Number of cameras detected: ", numCams)
if not numCams:
    print("Insufficient number of cameras. Exiting...")
    exit()

Number of cameras detected:  2


In [7]:
def enableEmbeddedTimeStamp(cam, enableTimeStamp):
    embeddedInfo = cam.getEmbeddedImageInfo()
    if embeddedInfo.available.timestamp:
        cam.setEmbeddedImageInfo(timestamp = enableTimeStamp)
        if(enableTimeStamp):
            print("\nTimeStamp is enabled.\n")
        else:
            print("\nTimeStamp is disabled.\n")

In [8]:
# open qtconsole if it's not already running
try:
    console_is_running
except NameError:
    console_is_running = True
    %qtconsole

In [9]:
def printCameraInfo(cam):
    camInfo = cam.getCameraInfo()
    print("\n*** CAMERA INFORMATION ***\n")
    print("Serial number - ", camInfo.serialNumber)
    print("Camera model - ", camInfo.modelName)
    print("Camera vendor - ", camInfo.vendorName)
    print("Sensor - ", camInfo.sensorInfo)
    print("Resolution - ", camInfo.sensorResolution)
    print("Firmware version - ", camInfo.firmwareVersion)
    print("Firmware build time - ", camInfo.firmwareBuildTime)
    fRateProp = cam.getProperty(fc2.PROPERTY_TYPE.FRAME_RATE)
    print("FrameRate - ", fRateProp.absValue)
    print()

In [10]:
c = fc2.Camera()
c.connect(bus.getCameraFromIndex(0))
printCameraInfo(c)

d = fc2.Camera()
d.connect(bus.getCameraFromIndex(1))
printCameraInfo(d)


*** CAMERA INFORMATION ***

Serial number -  16060682
Camera model -  b'Chameleon3 CM3-U3-13Y3C'
Camera vendor -  b'Point Grey Research'
Sensor -  b'OnSemi PYTHON1300 (1/2" Color CMOS)'
Resolution -  b'1280x1024'
Firmware version -  b'1.9.3.0'
Firmware build time -  b'Mon Jul 27 20:45:52 2015'
FrameRate -  92.39653778076172


*** CAMERA INFORMATION ***

Serial number -  16061221
Camera model -  b'Chameleon3 CM3-U3-13Y3C'
Camera vendor -  b'Point Grey Research'
Sensor -  b'OnSemi PYTHON1300 (1/2" Color CMOS)'
Resolution -  b'1280x1024'
Firmware version -  b'1.9.3.0'
Firmware build time -  b'Mon Jul 27 20:45:52 2015'
FrameRate -  92.39653778076172



In [11]:
def img2array(image):
    return(np.array(image.getData(), dtype="uint8").reshape( (image.getRows(), image.getCols()) ))

In [22]:
# start recording video
def saveAviHelper2(cam, cam2, fileFormat, fileName, fileName2, frameRate, maxImgs = 500):
    
    numImages = 0

    avi = fc2.AVIRecorder()
    avi2 = fc2.AVIRecorder()

    for i in range(maxImgs):
        
        try:
            image = cam.retrieveBuffer()
            image2 = cam2.retrieveBuffer()
        except fc2.Fc2error as fc2Err:
            print("Error retrieving buffer : ", fc2Err)
            continue

        print("Grabbed image {}".format(i))

        if (i == 0):
            if fileFormat == "AVI":
                avi.AVIOpen(fileName, frameRate)
                avi2.AVIOpen(fileName2, frameRate)
#             elif fileFormat == "MJPG":
#                 avi.MJPGOpen(fileName, frameRate, 75)
#             elif fileFormat == "H264":
#                 avi.H264Open(fileName, frameRate, image.getCols(), image.getRows(), 1000000)
            else:
                print("Specified format is not available.")
                return
            
            # show still image
            img = np.concatenate((img2array(image), img2array(image2)), axis = 1)

            # Display the resulting frame
            cv2.imshow('image', img)

        # break when "q" is pressed on keyboard
        if cv2.waitKey(1) & 0xFF == ord('q'):
            for jj in range(10):
                cv2.destroyAllWindows()
            break


        avi.append(image)
        avi2.append(image2)
        numImages += 1
        print("Appended image {}...".format(i))

    print("Appended {} images to {} file: {}...".format(numImages, fileFormat, fileName))
    avi.close()
    avi2.close()

In [23]:
c = fc2.Camera()
c.connect(bus.getCameraFromIndex(0))
enableEmbeddedTimeStamp(c, True)
c.startCapture()


d = fc2.Camera()
d.connect(bus.getCameraFromIndex(1))
enableEmbeddedTimeStamp(c, True)
d.startCapture()



TimeStamp is enabled.


TimeStamp is enabled.



In [24]:
# display images via live preview

def livePreview2():
    cv2.namedWindow('image',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('image', 1000,800)
    while(True):
        stt = time.time()
        # Capture frame-by-frame
        image = c.retrieveBuffer()
        image2 = d.retrieveBuffer()
        img = np.concatenate((img2array(image), img2array(image2)), axis = 1)

        # Our operations on the frame come here
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Display the resulting frame
        cv2.imshow('image', img)
        
        print(time.time() - stt)

        # break when "q" is pressed on keyboard
        if cv2.waitKey(1) & 0xFF == ord('q'):
            for jj in range(10):
                cv2.destroyAllWindows()
            break



In [20]:
# livePreview2()

0.3120591640472412
0.278092622756958
0.2782113552093506
0.2815732955932617
0.2824079990386963
0.28461670875549316
0.2845187187194824
0.2802093029022217
0.28780293464660645
0.2887556552886963
0.2811713218688965
0.2767772674560547
0.28648996353149414
0.28078174591064453
0.3092641830444336
0.3320889472961426
0.3278653621673584
0.33365511894226074
0.3318319320678711
0.31677722930908203
0.3274500370025635
0.37836408615112305
0.354494571685791
0.31917619705200195
0.2864804267883301
0.27871251106262207
0.2787809371948242


In [63]:
directory = os.path.join("C:\\Users\\Combes4\Desktop\\twoCamSave2")

movieID = re.sub(":", "_", str(datetime.datetime.now())) 
fileName = os.path.join(directory, "SaveImageToAviEx1" + movieID +".avi")
fileName2 = os.path.join(directory, "SaveImageToAviEx2" + movieID +".avi")
saveAviHelper2(c,d, "AVI", fileName.encode("utf-8"), fileName2.encode("utf-8"), 10)

Grabbed image 0
Appended image 0...
Grabbed image 1
Appended image 1...
Grabbed image 2
Appended image 2...
Grabbed image 3
Appended image 3...
Grabbed image 4
Appended image 4...
Grabbed image 5
Appended image 5...
Grabbed image 6
Appended image 6...
Grabbed image 7
Appended image 7...
Grabbed image 8
Appended image 8...
Grabbed image 9
Appended image 9...
Grabbed image 10
Appended image 10...
Grabbed image 11
Appended image 11...
Grabbed image 12
Appended image 12...
Grabbed image 13
Appended image 13...
Grabbed image 14
Appended image 14...
Grabbed image 15
Appended image 15...
Grabbed image 16
Appended image 16...
Grabbed image 17
Appended image 17...
Grabbed image 18
Appended image 18...
Grabbed image 19
Appended image 19...
Grabbed image 20
Appended image 20...
Grabbed image 21
Appended image 21...
Grabbed image 22
Appended image 22...
Grabbed image 23
Appended image 23...
Grabbed image 24
Appended image 24...
Grabbed image 25
Appended image 25...
Grabbed image 26
Appended image 

In [ ]:
# When everything done, release the capture
c.stopCapture()
c.disconnect()

d.stopCapture()
d.disconnect()
cv2.destroyAllWindows()

In [64]:
os.getcwd()

'C:\\Users\\Combes4\\Desktop\\twoCamSave2\\2018-01-24 11_45_24.297750'

In [65]:
fileName

'C:\\Users\\Combes4\\Desktop\\twoCamSave2\\SaveImageToAviEx12018-01-24 11_57_54.601993.avi'

In [66]:
# combine videos to see if they're both the same

vidPath =  re.sub(".avi", "-0000.avi", fileName)
vidPath2 = re.sub(".avi", "-0000.avi", fileName2)

cap = cv2.VideoCapture(vidPath)
cap2 = cv2.VideoCapture(vidPath2)

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

print( length, width, height, fps)

224 1280 1024 10.0


In [67]:

ctr = 0
photoDir = os.path.join("C:\\Users\\Combes4\Desktop\\twoCamSave2\\", movieID)

# make directory, if it doesn't exist
if not os.path.exists(photoDir):
    os.makedirs(photoDir)

    
    #refref" speed this up with opencv
for ff in range(length):
    ret, frame = cap.read()
    ret2, frame2 = cap2.read()
    im2 = np.concatenate((frame, frame2), axis = 1)
    io.imsave(os.path.join(photoDir, str(ctr).zfill(4) + '.png'), im2)
    print(ff)
    ctr += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223


In [68]:
# save images as video
from subprocess import call
os.chdir(photoDir)

call(["ffmpeg", "-start_number", "0", "-i", "%04d.png", "-r", "10",  "-c:v", "h264", "-pix_fmt", "yuv420p", "-y", "combinedVid.mp4"]) 

0